In [136]:
import os

In [137]:
%pwd 

'/Users/sultanalyami'

In [138]:
os.chdir('../')

In [139]:
%pwd

'/Users'

In [73]:
from dataclasses import dataclass
from pathlib import Path
# Training the model data class 
@dataclass(frozen=True)
class TrainingConfig:
    root_dir : Path 
    trained_model_path: Path
    update_base_model_path: Path
    training_data : Path
    params_is_augmentation : bool 
    params_image_size : list
    params_batch_size : int
    params_epochs : int     
    params_learning_rate : float



# Callbacks data class 

@dataclass(frozen=True)
class PreaperModelCallBacks:
    root_dir : Path 
    tensrboard_root_log_dir: Path
    checkpoint_model_filepath: Path


In [74]:
from srcTrackVision.utils.common import read_yaml , create_directories

In [75]:
class ConfigurationManager:
    def __init__(
            self , 
            config_filepath = '/Users/sultanalyami/Desktop/TrackVision_Project/TrackVision/config/config.yaml',
            params_filepath = '/Users/sultanalyami/Desktop/TrackVision_Project/TrackVision/params.yaml' ):
        

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        # Create the root dir 
        create_directories([self.config.artifacts_root])



    def get_preaper_model_callbacks_config(self) -> PreaperModelCallBacks:
        config = self.config.preaper_callbacks
        model_chpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_chpt_dir) ,
            Path(config.tensrboard_root_log_dir)
        ])
        preaper_model_callbacks_config = PreaperModelCallBacks(
            root_dir= Path(config.root_dir),
            tensrboard_root_log_dir = Path(config.tensrboard_root_log_dir),
            checkpoint_model_filepath = Path(config.checkpoint_model_filepath)
             

        )

    
        return preaper_model_callbacks_config
    

    def get_training_config(self) -> TrainingConfig:
        training =  self.config.training
        prepare_base_model = self.config.preaper_base_model 
        params = self.params
        train_data = os.path.join(self.config.data_ingestion.unzip_file, "archive/CK+48")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir = Path(training.root_dir),
            trained_model_path= Path(training.trained_model_path),
            update_base_model_path= Path(prepare_base_model.update_base_model_path),
            training_data = Path(train_data) ,
            params_is_augmentation = params.AUGMENTATION, 
            params_image_size = params.IMAGE_SIZE,
            params_batch_size = params.BATCH_SIZE,
            params_epochs = params.EPOCHS,
            params_learning_rate = params.LEARNING_RATE
        )

        return training_config

In [76]:
import time 
import tensorflow as tf 

In [77]:
class PreaperCallBacks:
    def __init__ (self  , config = PreaperModelCallBacks):
        self.config = config

    @property
    def _create_tb_callbacks (self):
        timestamp = time.strftime('%Y-%m-%d-%H-%M-%S')
        tb_running_log_dir = os.path.join(self.config.tensrboard_root_log_dir  
                                          , f'tb_logs_at_{timestamp}')
        

        return tf.keras.callbacks.TensorBoard(log_dir = tb_running_log_dir)
    
    @property
    def _create_checkpoint_callbacks (self):
        return tf.keras.callbacks.ModelCheckpoint(filepath= self.config.checkpoint_model_filepath, 
                                                   save_best_only= True
        )
    

    def get_tb_checkpoint_callbacks (self):
        return [
            self._create_tb_callbacks,
            self._create_checkpoint_callbacks
        ]

In [123]:
class Training:
    def __init__(self , config = TrainingConfig):
        self.config = config


    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.update_base_model_path
        )
    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.30
        )  

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = 'bilinear',
        )

        valid_generator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )


        self.valid_generator = valid_generator.flow_from_directory(
                directory= self.config.training_data
                , subset= 'validation'
                , shuffle= False
                , **dataflow_kwargs
                
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                horizontal_flip=False 
                , rotation_range = 10
                , zoom_range= 0.1
                , **datagenerator_kwargs

            )
        else : 
            train_datagenerator = valid_generator
                        
        self.train_generator = train_datagenerator.flow_from_directory(
                        directory= self.config.training_data
                        , shuffle= True
                        , subset= 'training'
                        , **dataflow_kwargs
        )
    @staticmethod 
    def save_model(path : Path  , model : tf.keras.models.Model):
        model.save(path)


    def train(self , callbacks_list : list):
        self.steps_per_epoch = self.train_generator.samples // self.config.params_batch_size
        self.validation_steps = self.valid_generator.samples // self.config.params_batch_size

        self.model.fit(
                    self.train_generator
                    , steps_per_epoch = self.steps_per_epoch
                    , validation_steps = self.validation_steps
                    , validation_data = self.valid_generator
                    , epochs = self.config.params_epochs
                    , callbacks = callbacks_list
        ) 

        self.save_model(
           path = self.config.trained_model_path
           , model = self.model
           
        )


        

In [126]:
from srcTrackVision import logger
try:
    # configuration for callbacks
    config = ConfigurationManager()
    preaper_callbacks_config = config.get_preaper_model_callbacks_config()
    prepaper_model_callbacks = PreaperCallBacks(config= preaper_callbacks_config)
    callback_list =  prepaper_model_callbacks.get_tb_checkpoint_callbacks()
    # configuration for model training 
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callbacks_list= callback_list
    
    )
    

except Exception as e:
    logger.exception(e)
    raise e

2023-06-17 20:32:30,309 - trackvisionlogger - INFO - yaml file /Users/sultanalyami/Desktop/TrackVision_Project/TrackVision/config/config.yaml loaded successfully
2023-06-17 20:32:30,312 - trackvisionlogger - INFO - yaml file /Users/sultanalyami/Desktop/TrackVision_Project/TrackVision/params.yaml loaded successfully
2023-06-17 20:32:30,313 - trackvisionlogger - INFO - Created directory at: /Users/sultanalyami/Desktop/TrackVision_Project/TrackVision/artifacts
2023-06-17 20:32:30,316 - trackvisionlogger - INFO - Created directory at: /Users/sultanalyami/Desktop/TrackVision_Project/TrackVision/artifacts/checkpoint_dir
2023-06-17 20:32:30,316 - trackvisionlogger - INFO - Created directory at: /Users/sultanalyami/Desktop/TrackVision_Project/TrackVision/artifacts/preaper_callbacks/tensorboard_log_dir
2023-06-17 20:32:30,319 - trackvisionlogger - INFO - Created directory at: /Users/sultanalyami/Desktop/TrackVision_Project/TrackVision/artifacts/training
Found 292 images belonging to 7 classes.
